<a href="https://colab.research.google.com/github/Dhanush-sai-reddy/langchain/blob/main/1semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain-community pypdf

In [ ]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [ ]:
from langchain_core.documents import Document
#sample from docs
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "FLAP-SAM.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

In [ ]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

In [ ]:
#splittin
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=880,
    chunk_overlap=140,##600
    add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

print(len(all_splits))

In [ ]:
#embeddings

In [ ]:
pip install -U langchain-google-genai google-generativeai


In [ ]:
pip install -U "langchain-openai"

In [ ]:
pip install -U langchain-huggingface sentence-transformers


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"

)

vector = embeddings.embed_query(
    "Federated learning with SAM for 3D segmentation"
)

print(len(vector))


In [ ]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

In [ ]:
vector_store = InMemoryVectorStore(embeddings)#we specify to use this embeddings model to convert texts

In [ ]:
ids = vector_store.add_documents(documents=all_splits)

In [ ]:
ids[0]#identifier

In [ ]:
results = vector_store.similarity_search(
    "segmentation"
)

print(results[3])

In [ ]:

results = vector_store.similarity_search_with_score("What is LoRA")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

In [ ]:
embedding = embeddings.embed_query("dataset used")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

####well here we move to retrivers part

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(
    [
        "Cost reduction by FLAP-SAM",
        "what  is Lora?",
    ],
)[0]
#since we used a hugging face embeddings model of 384 d,it may not work well as /for a semantic search